In [ ]:
# stdlib
import random
import time

# third party
import veilid

# first party
from veilid_streamer import VeilidStreamer

In [ ]:
host = "localhost"
port = 5960

In [ ]:
async def noop_callback(update: veilid.VeilidUpdate):
    pass


conn = await veilid.json_api_connect(host, port, noop_callback)

In [ ]:
router = await (await conn.new_routing_context()).with_default_safety()
dht_key_str = input("Enter DHT Key of the receiver: ")
dht_key = veilid.TypedKey(dht_key_str.lstrip("VLD0:"))
try:
    await router.close_dht_record(key=dht_key)
except veilid.VeilidAPIErrorGeneric:
    pass
await router.open_dht_record(key=dht_key, writer=None)
record_value = await router.get_dht_value(key=dht_key, subkey=0, force_refresh=True)
private_route = await conn.import_remote_private_route(record_value.data)

In [ ]:
vs = VeilidStreamer(connection=conn, router=router)

for message_size_kb in range(0, 13):  # Powers of two from 1 to 4096
    message_size_kb = 2**message_size_kb
    current_time = int(time.time()).to_bytes(4, byteorder="big")  # 4 bytes
    message_size = (message_size_kb * 1024) - 4
    message = current_time + random.randbytes(message_size)
    print(f"Sending message of size {len(message)} bytes")
    await vs.stream(private_route, message)